In [6]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
import MMJfunc

In [7]:
def get_bearing(point1, point2):
    # this code calculate the bearing of any given pair of longitude, latitude  
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(point1[0], point1[1], point2[0], point2[1])
    return fwd_azimuth

def edge_bearing(edge):
    # this function calculate the bearing from the starting and ending node of each road segment
    bearing = get_bearing(edge[0], edge[len(edge) - 1])
    return(bearing)
    
def conv_angle(angle):
    # this function convert angle from -pi,pi to 0,2*pi
    if angle < 0 :
        angle = angle + 360
    return(angle)

def adjust_angle(angle):
    # this function converts the angle so that if the angle is greater than pi it replaced with 2pi-angle
    if angle > 180:
        angle = 360 - angle
    return angle

def conc(a):
    #function to convert list or integer in osmid into a unique string id 
    if type(a) is int:
        return str(a)
    ans = ",".join(map(str, a))
    return ans

def err_polygon(curr_loc, err_size):
    # function that output shapely polygon for point error bound
    x = curr_loc['geometry'].iloc[0].x
    y = curr_loc['geometry'].iloc[0].y
    
    err_coord = [[x - err_size, y + err_size], 
                 [x + err_size, y + err_size],
                 [x + err_size, y - err_size],
                 [x - err_size, y - err_size]]

    poly_coord = Polygon(err_coord)
    # #print(ply_coord)
    df = {'Attribute' : ['name1'], 'geometry':poly_coord}

    #projected to UTM 31 
    err_poly = gpd.GeoDataFrame(df, geometry = 'geometry', crs = "EPSG:32631")
    
    return err_poly

def point_matching(curr_loc, curr_edge):
    # matched position to the current edge 
    # input need to be panda series
    # curr loc need attribute geometry point
    # curr_edge need attribute 'geometry' lines
    # output a point that matched to the current edge
    dist = curr_edge['geometry'].project(curr_loc['geometry']).iloc[0]
    matched_point = list(curr_edge['geometry'].interpolate(dist).coords)
    matched_point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([matched_point[0][0]], [matched_point[0][1]]), crs="EPSG:32631")
    return matched_point

def check_connect(link):
    # this function check connectivity of candidate link with the current edge
    # needed to convert key to column and the previous end node saved in the dataframe
    # 1 if its connect 0 if elsewhere
    if (link['u'] == link['prev_end_node']):
        a = 1
    else : 
        a = 0
    return a

In [8]:
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
nodes_utm = pd.read_pickle('nodes_utm.pkl')
curr_edge = pd.read_pickle('current_edge.pkl')

# for debugging purposes start at iteration point 14 
point_index = 13
# this is for debugging purposes, true 
prev_loc = gdf_utm.iloc[point_index-1].to_frame().T
last_matched = point_matching(prev_loc, curr_edge.iloc[0])


# current location 
curr_loc = gdf_utm.iloc[[point_index]]

# find longitude and latitude for last matched data
last_matched['lon_lat'] = last_matched.to_crs({'init': 'epsg:4326'})

C:\Users\GRIPS\anaconda3\envs\mitsubishi-a\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [9]:
err_size = 38
# extract node id
edge_node1 = nodes_utm.loc[curr_edge.index[0][0]]
edge_node2 = nodes_utm.loc[curr_edge.index[0][1]]

edge_link = []

# select the end node 
if (curr_loc['GPS Bearing'].iloc[0] > 0 and curr_loc['GPS Bearing'].iloc[0] < 180):
    # if object movint to the right of map select the one with larger longitude
    if edge_node1.x > edge_node2.x:
        end_node = edge_node1
        start_node = edge_node2
    else:
        end_node = edge_node2
        start_node = edge_node1
else:
    # if object is moving to the left select node with smaller longitude
    if edge_node1.x < edge_node2.x:
        end_node = edge_node1
        start_node = edge_node2
    else:
        end_node = edge_node2
        start_node = edge_node1

# get edges inside the error region 
err_poly = err_polygon(curr_loc, err_size)

intersects = gpd.sjoin(err_poly, edges_utm, predicate='intersects')
contains = gpd.sjoin(err_poly, edges_utm, predicate='contains')

if (len(intersects) + len(contains)) >0:
    # extract index from edges that intersect with error polygon 
    int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
    # extract index from edges that contained in the error polygon 
    cont_index = contains[['index_right0', 'index_right1', 'index_right2']]

    # merge index
    index = pd.concat([int_index, cont_index])
    # drop duplicate
    index = index.drop_duplicates()

    # initialize candidate edges 
    appended_edge = []

    # extract candidate eges  
    for i in range(len(index)):
        edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
        appended_edge.append(edge_list)

    candidate_link = edges_utm.loc[appended_edge]
    # store previous end link info for connectivity checking 
    candidate_link['prev_end_node'] = np.repeat(end_node.name, len(candidate_link))
    # put u and v into column for connectivity checking
    candidate_link_uv = candidate_link.reset_index()
    
    # calculate perpendicular distance 
    # initialize list that hold perpendicular distance between points and edges
    p_dist = []
    # initialize list that hold connectivity 
    conn = []
    # calculate perpendicular distance between current point and connectivity
    for i in range(len(candidate_link)):
        p_dist.append(candidate_link['geometry'].iloc[i].distance(curr_loc['geometry']).iloc[0])
        conn.append(check_connect(candidate_link_uv.iloc[i]))
    
    # attach perpendicular distance to candidate link 
    candidate_link['perp_dist'] = p_dist
    
    # attach connectivity 
    candidate_link['connectivity'] = conn

    # print(candidate_link)

    # calculate heading error
    # convert lat lon into tupple coordinate 
    candidate_link['lon_lat_pair'] = candidate_link.lon_lat.apply(lambda geom: list(geom.coords))

    # calculate bearing frome start and end node for each candidate link (see notes below)
    bearing_raw = candidate_link['lon_lat_pair'].apply(edge_bearing)

    # convert bearing from -pi, pi to 0, 2pi range
    candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)

    # heading difference = abs(gps heading - edge bearing)
    heading_diff = abs(candidate_link['edge_heading'] - curr_loc['GPS Bearing'].iloc[0])
            
    # convert heading difference so that all its values lie from 0 to pi because the contribution of angle x and 2pi-x should be equal.
    candidate_link['heading_error'] = heading_diff.apply(adjust_angle)    
    
    # initialize input for MMJ
    PD = candidate_link['perp_dist'].to_list()
    HE = candidate_link['heading_error'].to_list()
    TR = candidate_link['connectivity'].to_list()
    
    mmj_res = MMJfunc.MMJ(PD, HE, TR, 0)
    edge_link.append(candidate_link['osmid'].iloc[mmj_res])
    


In [11]:
mmj_res

3

In [10]:
candidate_link

,,,osmid,lanes,ref,name,highway,maxspeed,oneway,reversed,length,geometry,width,access,lon_lat,str_id,prev_end_node,perp_dist,connectivity,lon_lat_pair,edge_heading,heading_error
u,v,key,,,,,,,,,,,,,,,,,,,,
53200191,135535395,0,"[673214248, 234046511]",3,L 793,Wolbecker Straße,secondary,50,False,True,54.889,"LINESTRING (819585.705 5766245.100, 819590.522...",NaN,NaN,"LINESTRING (7.65181 51.95487, 7.65188 51.95486...","673214248,234046511",53200191,3.720433,1,"[(7.651807, 51.9548654), (7.6518768, 51.954862...",93.997786,134.097792
135535395,53200191,0,"[673214248, 234046511]",3,L 793,Wolbecker Straße,secondary,50,False,False,54.889,"LINESTRING (819640.807 5766244.782, 819631.704...",NaN,NaN,"LINESTRING (7.65261 51.95483, 7.65247 51.95483...","673214248,234046511",53200191,3.720433,0,"[(7.6526059, 51.9548309), (7.6524737, 51.95483...",273.998415,45.901579
6152925224,53200191,0,470244584,3,L 793,Wolbecker Straße,secondary,50,False,True,36.973,"LINESTRING (819548.588 5766245.690, 819569.082...",NaN,NaN,"LINESTRING (7.65127 51.95489, 7.65157 51.95488...",470244584,53200191,4.941025,0,"[(7.6512692, 51.954892), (7.6515661, 51.954876...",94.576660,134.676666
53200191,6152925224,0,470244584,3,L 793,Wolbecker Straße,secondary,50,False,False,36.973,"LINESTRING (819585.705 5766245.100, 819574.575...",NaN,NaN,"LINESTRING (7.65181 51.95487, 7.65165 51.95487...",470244584,53200191,4.941025,1,"[(7.651807, 51.9548654), (7.6516457, 51.954873...",274.577083,45.322911


In [21]:
edge_link

[470244584]